In [1]:
import enum
import typing
import abc
import itertools
import dataclasses
from pathlib import Path
from types import SimpleNamespace

import attrs
import numpy as np
from IPython import display

from arc25.dsl import types, primitives
from arc25.dsl import api as dsl
from arc25 import tools, dataset, sandbox, symmetry, prompts

In [2]:
proj_root = Path("..").resolve()
data_path = proj_root / "data"
db_root = data_path / "solutions"


In [3]:
ds = await dataset.Dataset.from_binary(data_path / "all-challenges.cbor.xz")
len(ds.challenges), sorted(ds.subsets)

(1307,
 ['all',
  'arc-agi-1',
  'arc-agi-1.evaluation',
  'arc-agi-1.training',
  'arc-agi-2',
  'arc-agi-2.evaluation',
  'arc-agi-2.training',
  'concept-arc',
  'harc',
  'harc.evaluation',
  'harc.training',
  'larc'])

In [4]:
solutions = await dataset.SolutionDB.load(db_root)
sol_with_code = sorted(k for k,v in solutions.solutions.items() if v.rule)
print(len(solutions.solutions), len(sol_with_code))
print(sol_with_code)

46 37
['00576224', '36fdfd69', '3af2c5a8', '4612dd53', '46442a0e', '54d82841', '5614dbcf', '642d658d', '6e19193c', '6f8cd79b', '72322fa7', '7bb29440', '8eb1be9a', '97999447', '9841fdad', '9f5f939b', 'a740d043', 'bf699163', 'ca-00-01', 'ca-00-10', 'ca-03-02', 'ca-03-04', 'ca-05-03', 'ca-08-01', 'ca-09-01', 'ca-10-04', 'ca-12-02', 'ca-14-05', 'ca-14-06', 'ca-15-10', 'cbded52d', 'ce039d91', 'd931c21c', 'e1d2900e', 'ea32f347', 'f9d67f8b', 'fd02da9e']


In [5]:
ckey = "ca-03-02"
chal = ds.challenges[ckey]
sol = solutions.solutions[ckey]
resp = prompts.parse_explanation(sol)
display.display(display.Markdown(tools.solution_to_markdown(resp)))


## Description
### Hypothesis
- <span style="background-color:rgba(0,0,0,0.15)">Black</span> background.
- One horizontal and one vertical colored dotted line.
- Impossible to say which of the two is painted over the other.

## Rule
- Start either from the input, or a fully <span style="background-color:rgba(0,0,0,0.15)">black</span> canvas of the same shape.
- First, paint a solid vertical line at the position and in the color of the dotted vertical line in the input.
- Second, paint a solid horizontal line at the position and in the color of the dotted horizontal line in the input.
## Plan
- There are no distractions, so we can determine the line positions simply from the count of foreground cells.

```python
def solution(input: Image) -> AnyImage:
    output = input
    h,w = input.shape
    # identify foreground cells
    fg_mask = ~mask_color(input, BLACK)
    # paint all columns with more than one foreground cell
    # (there will be just one)
    for col in range(w):
        col_mask = mask_col(input, col)
        mask = col_mask & fg_mask
        if mask.count()>1:
            c, = most_common_colors(apply_mask(input, mask))
            output = fill(output, c, clip=col_mask)
    # same for the row
    for row in range(h):
        row_mask = mask_row(input, row)
        mask = row_mask & fg_mask
        if mask.count()>1:
            c, = most_common_colors(apply_mask(input, mask))
            output = fill(output, c, clip=row_mask)
    return output
```

In [25]:
enc = prompts.PromptEncoder()

print(enc.encode_response(resp))

<descr>hypothesis
- Black background.
- One horizontal and one vertical colored dotted line.
- Impossible to say which of the two is painted over the other.
</descr>
<rule>
- Start either from the input, or a fully black canvas of the same shape.
- First, paint a solid vertical line at the position and in the color of the dotted vertical line in the input.
- Second, paint a solid horizontal line at the position and in the color of the dotted horizontal line in the input.
</rule>
<plan>
- There are no distractions, so we can determine the line positions simply from the count of foreground cells.
</plan>
<impl>```python
def solution(input: Canvas) -> Canvas:
    output = input
    h,w = input.shape
    # identify foreground cells
    fg_mask = ~mask_color(input, BLACK)
    # paint all columns with more than one foreground cell
    # (there will be just one)
    for col in range(w):
        col_mask = mask_col(input, col)
        mask = col_mask & fg_mask
        if mask.count()>1:
      

In [8]:
print(enc.encode_response(prompts.ReasonedSolution(
    descr=dict(inputs="...",hypothesis="..."),
    rule_descr="...",
    impl_plan_descr="...",
    rule_impl="""
def solution(input: Canvas) -> Canvas:
    # TODO: implement rule
    return output
    """
)))

<descr>inputs
...
</descr>
<descr>hypothesis
...
</descr>
<rule>
...
</rule>
<plan>
...
</plan>
<impl>```python
def solution(input: Canvas) -> Canvas:
    # TODO: implement rule
    return output
```</impl>
